In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd

from tensorflow import keras
from tensorflow.keras.datasets import mnist

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical

from keras.callbacks import ModelCheckpoint

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# mnist 데이터 불러오기

mnist = keras.datasets.mnist

In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
x_train.shape

(60000, 28, 28)

In [ ]:
y_train.shape

(60000,)

In [ ]:
np.random.seed(1234)
val_ind = np.random.choice(x_train.shape[0], 10000, replace=False)
tr_ind = np.setdiff1d(np.arange(x_train.shape[0]),val_ind)

x_valid, y_valid = x_train[val_ind], y_train[val_ind]
x_train, y_train = x_train[tr_ind], y_train[tr_ind]

In [ ]:
x_train = (x_train / 255) - 0.5
x_valid = (x_valid / 255) - 0.5
x_test = (x_test / 255) - 0.5

x_train = np.expand_dims(x_train, axis=3)
x_valid = np.expand_dims(x_valid, axis=3)
x_test = np.expand_dims(x_test, axis=3)

y_train_onehot = to_categorical(y_train, num_classes=10)
y_valid_onehot = to_categorical(y_valid, num_classes=10)
y_test_onehot = to_categorical(y_test, num_classes=10)

In [ ]:
print(x_train.shape)
print(x_valid.shape)
print(x_test.shape)

(50000, 28, 28, 1)
(10000, 28, 28, 1)
(10000, 28, 28, 1)


In [ ]:
filter_size = 3
pool_size = 2

model = Sequential([
  keras.Input(shape=(28,28,1)),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  MaxPooling2D(pool_size=pool_size),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 30, kernel_size = filter_size, padding='same'),
  BatchNormalization(),
  Activation('relu'),
  MaxPooling2D(pool_size=pool_size),
  Conv2D(filters = 30, kernel_size = (3,3), padding='same'),
  BatchNormalization(),
  Activation('relu'),
  Conv2D(filters = 30, kernel_size = filter_size, padding='same'),
  BatchNormalization(),
  Activation('relu'),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(50),
  Activation('relu'),
  Dense(10, activation='softmax'),
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 28, 28, 30)        300       
                                                                 
 batch_normalization_6 (Batc  (None, 28, 28, 30)       120       
 hNormalization)                                                 
                                                                 
 activation_7 (Activation)   (None, 28, 28, 30)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 28, 28, 30)        8130      
                                                                 
 batch_normalization_7 (Batc  (None, 28, 28, 30)       120       
 hNormalization)                                                 
                                                                 
 activation_8 (Activation)   (None, 28, 28, 30)       

In [ ]:
checkpoint_filepath = "/content/gdrive/best_cnn.ckpt"
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)


In [ ]:
# Compile the model.
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
history = model.fit(x=x_train,y=y_train_onehot,
          epochs=10,
          validation_data=(x_valid, y_valid_onehot),
          verbose=1,
          batch_size=256,
          callbacks=[model_checkpoint_callback]
)


Epoch 1/10
196/196 [==============================] - ETA: 0s - loss: 0.2576 - accuracy: 0.9278
Epoch 1: val_accuracy improved from -inf to 0.11680, saving model to /content/gdrive/MyDrive/AI_class/best_cnn.ckpt
196/196 [==============================] - 251s 1s/step - loss: 0.2576 - accuracy: 0.9278 - val_loss: 2.8895 - val_accuracy: 0.1168
Epoch 2/10
196/196 [==============================] - ETA: 0s - loss: 0.0436 - accuracy: 0.9868
Epoch 2: val_accuracy improved from 0.11680 to 0.54350, saving model to /content/gdrive/MyDrive/AI_class/best_cnn.ckpt
196/196 [==============================] - 235s 1s/step - loss: 0.0436 - accuracy: 0.9868 - val_loss: 1.1906 - val_accuracy: 0.5435
Epoch 3/10
196/196 [==============================] - ETA: 0s - loss: 0.0288 - accuracy: 0.9917
Epoch 3: val_accuracy improved from 0.54350 to 0.98630, saving model to /content/gdrive/MyDrive/AI_class/best_cnn.ckpt
196/196 [==============================] - 248s 1s/step - loss: 0.0288 - accuracy: 0.9917 - va

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
model.evaluate(x_test, y_test_onehot)

313/313 [==============================] - 11s 36ms/step - loss: 0.0272 - accuracy: 0.9915


[0.02715974487364292, 0.9915000200271606]

epoch을 10으로 하고 filter의 개수를 30으로 했을 때의 accuracy는 0.9915로 높게 나오는 편이다.
epoch 6에서 한 번 accuracy 업데이트가 멈췄다가 epoch 8에서 한번 더 업데이트 되는 것을 확인해볼 수 있다.